In [444]:
import pandas as pd
import numpy as np


In [445]:
date_columns = [
                'Account_Created_Date', 'Opportunity_Created_Date',
                'Quote_Expiry_Date', 'Last_Modified_Date',
                'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                ]

categorical_columns = [
        'Region', 'Territory', 'Bureaucratic_Code',
        'Source ', 'Billing_Country', 'Account_Name',
        'Opportunity_Name', 'Account_Owner', 'Opportunity_Owner',
        'Account_Type', 'Opportunity_Type', 'Quote_Type',
        'Delivery_Terms', 'Brand', 'Product_Type',
        'Size', 'Product_Category_B', 'Currency',
        'Last_Modified_By', 'Product_Family', 'Product_Name',
        'ASP_Currency', 'ASP_(converted)_Currency', 'Delivery_Quarter',
        'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Stage',
        'Prod_Category_A'
    ]

# Columnnas excluidas porque tienen igual valor en todos sus registros
empty = ['Actual_Delivery_Date', 'Last_Activity',
        'Submitted_for_Approval','Prod_Category_A']

In [446]:

def open_and_filter(dataset):
    
    column_types = { col:'category' for col in categorical_columns }
    

    
    # read_csv
    df = pd.read_csv(dataset, parse_dates=date_columns, dtype=column_types,
                     index_col='ID', na_values=['Other', 'NaT', 'None'],
                     usecols=lambda x: x not in empty)
    
    # Re-typing
    df['Sales_Contract_No'] = df['Sales_Contract_No'].fillna(0).astype(np.int64)
    df['Month'] = pd.to_datetime(df['Month'], format='%Y - %m')
    
    # Agruping regions 
    df.loc[((df.Region == "EMEA")&(df.Territory.str.contains("America"))), "Region"] = "Americas"
    
    return df
    


# Pre-procesamiento de los datos

El objetivo de esta etapa, es recibir los datos "crudos" y realizar procedimientos necesarios para filtrar features de poco valor y crear otros features que revelen información de importancia, para que los modelos de machine learning que luego los utilizarán en una etapa posterior, puedan ralizar un predicción mas precisa.

In [447]:
df1 = open_and_filter("rawdata/Train_TP2_Datos_2020-2C.csv")
df2 = open_and_filter("rawdata/Test_TP2_Datos_2020-2C.csv")

In [448]:
print(f"The train set has {df1.shape[0]} elements and {df1.shape[1]} features")
print(f"The train set has {df2.shape[0]} elements and {df2.shape[1]} features")

The train set has 16947 elements and 47 features
The train set has 2551 elements and 46 features


In [449]:
# Hypotesis: the features that contains more than 75% of NaN values 
#            do not contribute sustancial inforation
na_values_rate = df1.isna().sum()/len(df1)
na_values_rate = na_values_rate[na_values_rate>0]
na_values_rate

Territory                    0.294978
Source                       0.560394
Billing_Country              0.001593
Account_Type                 0.006609
Brand                        0.974686
Product_Type                 0.970673
Size                         0.965422
Product_Category_B           0.970732
Price                        0.978993
Currency                     0.947188
Quote_Expiry_Date            0.272910
ASP                          0.189355
ASP_(converted)              0.189355
Planned_Delivery_End_Date    0.004426
Total_Amount                 0.003481
dtype: float64

In [450]:
# Drop
to_drop = na_values_rate[na_values_rate > 0.75].index.to_list()

trainset = df1.drop(columns = to_drop)
testset = df2.drop(columns= to_drop)

text = "', '".join(to_drop)
print(f"Columns '{text}' dropped")

Columns 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency' dropped


Para este modelo entonce no se consideraran las columnas ``'Brand'``, ``'Product_Type'``, ``'Size'``, ``'Product_Category_B'``, ``'Price'``, ``'Currency'``

In [451]:
# El resto de columnas con NA_Values
contains_na = na_values_rate[na_values_rate <= 0.75].index.to_list()
na_values_rate[contains_na]

Territory                    0.294978
Source                       0.560394
Billing_Country              0.001593
Account_Type                 0.006609
Quote_Expiry_Date            0.272910
ASP                          0.189355
ASP_(converted)              0.189355
Planned_Delivery_End_Date    0.004426
Total_Amount                 0.003481
dtype: float64

In [452]:

def my_fill_na(df,columns):

    result = df.copy()
    dtypes = result[columns].dtypes
    cat = dtypes[dtypes == "category"].index.to_list()
    not_cat = dtypes[dtypes != "category"].index.to_list()
    
    for col in cat:
        if "Other" not in result[col].cat.categories:
            result[col].cat.add_categories("Other",inplace=True)
        result[col].fillna("Other",inplace= True)
    
    for col in not_cat:
        result[col].fillna(result[col].mean(),inplace= True)

    
    return result

In [453]:
trainset = my_fill_na(trainset,contains_na)
testset = my_fill_na(testset,contains_na)

In [454]:
na_train = trainset.isna().sum() > 0
na_test = testset.isna().sum() > 0

print(f"Restan valores nulos en trainset: {na_train.any()}")
print(f"Restan valores nulos en testset: {na_test.any()}")

Restan valores nulos en trainset: False
Restan valores nulos en testset: False


Ya filtramos las features con excesivos Nan values y rellenamos aquellas que su procentage de nan values es moderado, con valores predeterminados.

### Definición del target de los modelos

In [455]:
trainset["target"] = (trainset["Stage"] == "Closed Won").astype(int)
trainset[["Stage","target"]].head(10)

,Stage,target
ID,,
27761,Closed Lost,0
27760,Closed Won,1
27446,Closed Won,1
16808,Closed Lost,0
16805,Closed Lost,0
16802,Closed Lost,0
16799,Closed Lost,0
27455,Closed Won,1
24353,Closed Lost,0


### Definición de valores categóricos

En esta sección, lo que vamos a hacer es buscar valores categóricos de los features del set de datos de prueba que no hayan sido contemplados en el set de entrenamiento, que por lo consiguiente, el modelo de machine learning los va a desconocer.

Una vez identificados, los reemplazaremos con un valor genérico para "otros valores"

In [456]:
set_A = set(testset["Territory"].value_counts().index)
set_B = set(trainset["Territory"].value_counts().index)
for value in set_B: set_A.discard(value)
set_A

{'Andorra', 'Finland', 'Mongolia'}

In [457]:
official_values = trainset["Delivery_Quarter"].cat.categories.to_list()
test_values = testset["Delivery_Quarter"].cat.categories.to_list()
display(official_values)
display(test_values)

['Q1', 'Q2', 'Q3', 'Q4']

['Q1', 'Q2', 'Q3', 'Q4']

In [458]:
excluded = empty+to_drop+["Stage","Opportunity_Name"]
results = []
for column in categorical_columns:
    if column in excluded: continue
    
    official_values = trainset[column].cat.categories.to_list()
    test_values = testset[column].cat.categories.to_list()
    
    other_values = set(test_values)
    for value in official_values: other_values.discard(value)
    
    if len(other_values)>0:
        
        if not "Other" in testset[column].cat.categories.to_list():
            testset[column].cat.add_categories("Other",inplace=True)
        
        if not "Other" in trainset[column].cat.categories.to_list():
            trainset[column].cat.add_categories("Other",inplace=True)
            
        testset[column].replace({x:"Other" for x in other_values},inplace=True)
    
        results.append((column,len(other_values)))


In [459]:
others_df = pd.DataFrame(results,columns= ["column", "discarted values"])
others_df

,column,discarted values
0,Territory,3
1,Billing_Country,3
2,Account_Name,205
3,Opportunity_Owner,5
4,Last_Modified_By,8
5,Product_Family,20
6,Product_Name,50


In [460]:
display(round(others_df["discarted values"].describe(),2))
print(f"\nTotal test values discarted {others_df['discarted values'].sum()}, ",end="")
print(f"%{100*round(others_df['discarted values'].sum()/testset.size,3)} of total test data")

count      7.00
mean      42.00
std       73.81
min        3.00
25%        4.00
50%        8.00
75%       35.00
max      205.00
Name: discarted values, dtype: float64


Total test values discarted 294, %0.3 of total test data


La información descartada es muy poca con respecto al volumen del set. Más adelante veremos si es posible extraer información de esto

### One-hot encoding 
Realizaremos un dataset básico para poder correr el modelo por primera vez y observar los resultados.
Luego realizaremos mejoras e ingeniería de features para ver como se comporta el modelo.

In [461]:
from sklearn.preprocessing import OneHotEncoder 

In [462]:
print(f"El set de entrenamiento tiene {trainset.size} elementos")
print(f"El set de test tiene {testset.size} elementos")

El set de entrenamiento tiene 711774 elementos
El set de test tiene 102040 elementos


In [463]:
excluded = empty+to_drop
excluded.append("Opportunity_Name")
excluded.append("Stage")
excluded.append("target")
toEncode = set([col if col not in excluded else "" for col in categorical_columns])
toEncode.discard("")
toEncode = list(toEncode)

print("Features categóricos a encodear")
toEncode

Features categóricos a encodear


['Delivery_Quarter',
 'ASP_Currency',
 'Account_Owner',
 'Region',
 'Billing_Country',
 'Opportunity_Owner',
 'Product_Name',
 'Product_Family',
 'Quote_Type',
 'Territory',
 'Account_Type',
 'Source ',
 'ASP_(converted)_Currency',
 'Opportunity_Type',
 'Bureaucratic_Code',
 'Delivery_Terms',
 'Account_Name',
 'Last_Modified_By',
 'Total_Taxable_Amount_Currency',
 'Total_Amount_Currency']

In [464]:
enc = OneHotEncoder(drop='if_binary')
enc.fit(trainset[toEncode])
ohed = pd.DataFrame(enc.transform(trainset[toEncode]).toarray())
print(f"Shape: {ohed.shape}")
print(f"Size: {ohed.size}")
ohed.info()

Shape: (16947, 2718)
Size: 46061946
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Columns: 2718 entries, 0 to 2717
dtypes: float64(2718)
memory usage: 351.4 MB


Decidimos abandonar esta propuesta debido a la poca escalabilidad que tiene este método. Debido a las numerosas columnas categóricas con numerosos valores posibles cada una, necesitamos un total de ``2.723`` columnas para `16.947` entradas, lo cual nos deja un total de `46.146.681` valores en nuestra tabla.

### Binary encoding 

Este método nos permite encodear las features categóricas reduciendo considerablemente el dataset. Si suponemos que los 16.947 valores categóricos posibles se distribuyen uniformemente en las 7 columnas, entonces necesitaremos ``⌊log_2(16.947/7)⌋ + 1  = 9`` columnas nuevas por cada columna categórica original del dataset. En total ``9*7 = 63`` una propuesta cuestionable pero mucho mejor que la anterior. 

In [465]:
import math
class BinaryEncoding():
    
    def __init__(self):
        self.__encodings = {}
        
    def __make_encoding(self,name,categories):

        encoding = {}
        n_cols = int(math.log(len(categories),2))+1
        cols_names = [ name+"_"+str(x) for x in range(n_cols) ]

        for i in range(len(categories)):
            encoding[categories[i]] = list(f'{bin(i)[2:]}'.zfill(n_cols))

        return (encoding,cols_names)
    
    def encode_Series(self, serie, name,verbose=False):
        
        
        if not name in self.__encodings:

            categories = serie.cat.categories.to_list()
            self.__encodings[name] =  self.__make_encoding(name,categories)
        
        encoding,cols_names = self.__encodings[name]
        data = []
        indexs = []
        
        for index, value in serie.items():

            data.append(encoding[value])
            indexs.append(index)

        df_result = pd.DataFrame(data,columns=cols_names,index=indexs)
        return df_result,cols_names
    
    def getEncoding(self):
        return self.__encodings
    
    def encode_DataFrame(self, df, toEncode, verbose = False):

        full_encoded,columns = self.encode_Series(df[toEncode[0]],toEncode[0],verbose)
       
        for col in toEncode[1:]:
            
            encoding,col_names = self.encode_Series(df[col],col,verbose)
            full_encoded[col_names] = encoding

        if verbose: evaluate_encoding(df[toEncode],full_encoded)

        return full_encoded


In [466]:
"""
def encode_serie(serie,name):
    
    encoding = {}
    categories = serie.cat.categories.to_list()
    n_cols = int(math.log(len(categories),2))+1
    cols_names = [ name+"_"+str(x) for x in range(n_cols) ]

    for i in range(len(categories)):
        encoding[categories[i]] = list(f'{bin(i)[2:]}'.zfill(n_cols))
    
    data = []
    indexs = []
    
    for index, value in serie.items():
        data.append(encoding[value])
        indexs.append(index)
        
    df_result = pd.DataFrame(data,columns=cols_names,index=indexs)
    return df_result, cols_names
"""
[]+[]

[]

In [467]:
def evaluate_encoding(original,encoded):

    # Recuento de cada una de las combinaciones de 
    # la lista de features categóricos sin encodear
    count1 = original.value_counts().to_frame()[0].values

    # Recuento de cada una de las combinaciones de 
    # la lista de features categóricos ENCODEADOS
    count2 = encoded.value_counts().to_frame()[0].values

    # Comparación
    print("El encoding fue realizado correctamente: ", np.equal(count1,count2).all())

In [468]:
"""
def BinaryEncoding(df,toEncode,verbose = False):
    
    print("Shape prev",df[toEncode].shape)
    
    full_encoded,columns = encode_serie(df[toEncode[0]],toEncode[0])
    for col in toEncode[1:]:
        encoding,col_names = encode_serie(df[col],col)
        full_encoded[col_names] = encoding
        
    if verbose: evaluate_encoding(df[toEncode],full_encoded)
        
    return full_encoded
"""
[]+[]

[]

In [469]:
encoder = BinaryEncoding()
trainEncoded = encoder.encode_DataFrame(trainset,toEncode,verbose=True)
testEncoded = encoder.encode_DataFrame(testset,toEncode,verbose=True)

El encoding fue realizado correctamente:  True
El encoding fue realizado correctamente:  True


In [470]:
print(f"Train-encoded shape: {trainEncoded.shape}")
print(f"Train-encoded size: {trainEncoded.size}")
print("--------------------------------")
print(f"Test-encoded shape: {testEncoded.shape}")
print(f"Test-encoded size: {testEncoded.size}")

Train-encoded shape: (16947, 97)
Train-encoded size: 1643859
--------------------------------
Test-encoded shape: (2551, 97)
Test-encoded size: 247447


Pudimos encodear las columnas categóricas reduciendo notablemente el espacio. El cálculo de columnas utilizadas previo valía con la suposición de que todas las features tenían la misma cantidad de valores. Pero aún no cumpliendose, la diferencia de columnas es poca con respecto al resultado obtenido con One Hot Encoding

Terminamos este procesamiento de datos, para ver como un modelo de RandomForest se comporta frente a esto.

In [471]:
final_cols = list()
categorical_columns.append("target")
for col in trainset.columns.to_list():
    if not col in categorical_columns and not col in date_columns:
        final_cols.append(col)
print(f"Selected {len(final_cols)} features no categóricos")
final_cols

Selected 13 features no categóricos


['Pricing, Delivery_Terms_Quote_Appr',
 'Pricing, Delivery_Terms_Approved',
 'Bureaucratic_Code_0_Approval',
 'Bureaucratic_Code_0_Approved',
 'Opportunity_ID',
 'Sales_Contract_No',
 'ASP',
 'ASP_(converted)',
 'Month',
 'Delivery_Year',
 'TRF',
 'Total_Amount',
 'Total_Taxable_Amount']

In [472]:
finalTrain = trainset.loc[:,final_cols+["target"]]
finalTrain[trainEncoded.columns.to_list()] = trainEncoded
finalTrain.shape

(16947, 111)

In [473]:
finalTest = testset.loc[:,final_cols]
finalTest[testEncoded.columns.to_list()] = testEncoded
finalTest.shape

(2551, 110)

In [475]:
# Se eliminaron las columnas vacías o con valores iguales
# Se rellenaron nan_values con promedios para features numéricos y con "others" para categóricos
# Se eliminaron valores categóricos del set de test que no estan en el set de entrenamiento
# Se realizó binary encoding para todos las features categóricas
# Se agregó la columna "target" la cual tiene 1/0 según es "Closed Won" o no
# Se eliminó la columna "Stage"

path = "datasets/"
name = "first-rf-model"
finalTrain.to_csv(path+name+"-train.csv", index = False)
finalTest.to_csv(path+name+"-test.csv", index = False)